In [3]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.polynomial.polynomial import polyfit
from brian2 import ms, second, pA, nA, Hz, amp
import scipy.io as sio
import matplotlib as mpl
from scipy.stats import qmc

mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

from LongRangeModel import LongRangeModel
from Params import Params
from utils.cache import Cache

In [1]:

cache = Cache()

engine = qmc.PoissonDisk(d=2, radius=0.018, rng=np.random.default_rng())
sample = engine.random(2000)

get_cache = False
simulations = []

if(get_cache):
    simulations = dict(cache.get_figure_data("bifurcation"))
    sample = simulations["sample"]

for i, (p_sup, p_dp) in enumerate(sample):
    params = Params(simulation_time=2, dt=0.001, I_stim=350*pA, I_vig=60*pA, stimulus_start=0.3, stimulus_end=0.35, p_sup=p_sup, p_dp=p_dp)

    t_span_euler = np.arange(0, params.simulation_time, params.dt)

    if(not get_cache):
        model = LongRangeModel(params)
        result = model.run()
        simulations.append(result)

        rE = result[0][:, np.where(t_span_euler == 1.8)[0][0], 0]*Hz
        rE_early = result[0][:, np.where(t_span_euler == 0.2)[0][0], 0]*Hz
    else:
        rE = np.array(simulations[f"{i}"]["rEs"])[:, np.where(t_span_euler == 1.8)[0][0], 0]*Hz
        rE_early = np.array(simulations[f"{i}"]["rEs"])[:, np.where(t_span_euler == 0.2)[0][0], 0]*Hz

    colour = ""

    if(np.all(rE > 1*Hz)): colour = "red"
    elif(np.all(rE < 1*Hz)): colour = "blue"
    elif(np.any(rE_early > 1*Hz)): colour = "peru"
    else: colour = "green"

    plt.scatter(p_dp, p_sup, s=10, color=colour)

if(not get_cache):
    data = {"sample": sample}

    for i, sim in enumerate(simulations):
        data[f"{i}"] = {
            "rEs": sim[0],
            "rI": sim[1],
            "sNMDA": sim[2],
            "sAMPA": sim[3],
            "sGABA": sim[4],
        }

    cache.cache_figure_data(data, "bifurcation")

# print(p_sup, p_dp)
# plt.scatter(sample[:, 0], sample[:,1], s=20)
plt.show()

NameError: name 'Cache' is not defined